# Transfer learning

In this lab we will make use of pretrained models in order to boost performance on smaller datasets. For this experiment, we will be working with an AlexNet model pretrained on the Imagenet dataset in order to get a good accuracy score on the Caltech 101 dataset.

### Prerequisites

1. In order to perform the experiments, please download in advance the Caltech 101 dataset from https://drive.google.com/file/d/137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp/view
2. In the working directory please create a folder named 'dataset' and a subfolder named 'caltech101' within it. Extract the dataset in the subfolder. The overall folder structure should look as follows: dataset/caltech101/101_ObjectCategories.
3. Install the torchvision module using 'conda install torchvision' if you have not done so already.

In [1]:
from tqdm import tqdm
import numpy as np
import numpy.random as random
import torch
import torchvision
import warnings
import matplotlib.pyplot as plt
import typing as t
from torch import Tensor
from torch.utils.data import random_split
from torch.utils.data import DataLoader, Dataset
from torchvision.models import AlexNet_Weights


warnings.filterwarnings('ignore')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed = 42

gen = torch.Generator()
gen.manual_seed(seed)
random.seed(seed)

Firstly, we will load the AlexNet model architecture using torchvision. All available models with their respective parameters can be found at: https://pytorch.org/vision/stable/models.html

In [2]:
model = torchvision.models.alexnet()

In the first run we will just load the model architecture, without the pretrained weights. We can visualize the model architecture as follows:

In [3]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

Next, we will load the Caltech 101 dataset and apply the neccesary transformations on it. Afterwards, we will split the dataset into train, validation and test.

In this block of code, define the dataloaders for train, validation and test and try to iterate through the data. What happens? Try to fix the problem using a lambda transform: https://pytorch.org/vision/stable/transforms.html#generic-transforms

In [4]:
from torchvision.transforms.v2 import Compose, ToImage, ToDtype, Resize, Normalize, Lambda


dataset = torchvision.datasets.Caltech101(
    './dataset',
    transform = Compose([
        ToImage(),
        ToDtype(torch.float, scale=True),
        Resize((224, 224)),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
)

batch_size= 16
n_samples = len(dataset)
train_ds, val_ds, test_ds = random_split(dataset, [0.8, 0.1, 0.1], gen)

# define dataloaders for train, validation and test
# iterate through the dataloaders
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, generator=gen)
valid_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=True, generator=gen)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True, generator=gen)

With the dataset ready, it is now time to adapt the model architecture in order to fit our needs. Define a new classifier for the AlexNet model having the same structure, changing only the number of output neurons to 101.

In [5]:
model.classifier

Sequential(
  (0): Dropout(p=0.5, inplace=False)
  (1): Linear(in_features=9216, out_features=4096, bias=True)
  (2): ReLU(inplace=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=4096, out_features=4096, bias=True)
  (5): ReLU(inplace=True)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [6]:
import torch.nn as nn
from torch.nn import Dropout, Linear, ReLU


# Create a new classifier similar to AlexNet
model.classifier = torch.nn.Sequential(
    Dropout(p=0.5, inplace=False),
    Linear(in_features=9216, out_features=4096, bias=True),
    ReLU(inplace=True),
    Dropout(p=0.5, inplace=False),
    Linear(in_features=4096, out_features=4096, bias=True),
    ReLU(inplace=True),
    Linear(in_features=4096, out_features=101, bias=True)
)

### Training the model

Define an Adam optimizer with a learining rate of 1e-4 and a cross entropy loss. Afterwards, train the model for 2 epochs. Note the results

In [7]:

class Metrics(t.TypedDict):
    accuracy: t.List[float]
    loss: t.List[float]


class TrainHistory(t.TypedDict):
    train: Metrics
    valid: Metrics


def train_validate(model: nn.Module,
                   train_dl: DataLoader,
                   valid_dl: DataLoader,
                   epochs: int,
                   loss_fn: nn.Module,
                   optim: torch.optim.Optimizer) -> TrainHistory:
    # Track history
    history: TrainHistory = {
        'train': {
            'accuracy': [],
            'loss': [],
        },
        'valid': {
            'accuracy': [],
            'loss': [],
        }
    }

    # Do Training & Validation & Testing
    for epoch in range(epochs):
        print('Epoch [%d/%d]' % (epoch + 1, epochs), end=' - ')

        ### Training ###
        model.train(True)
        model.requires_grad_(True)

        # Track across a single epoch
        train_loss = []
        train_accuracy = []

        for b, (X, y) in enumerate(train_dl):
            X, y = X.to(device), y.to(device)

            # Prevent grad accumulation
            optim.zero_grad()

            # Forward pass
            logits = model.forward(X)
            loss: Tensor = loss_fn(logits, y)
            y_pred: Tensor = logits.argmax(dim=1).detach()

            # Backward pass
            loss.backward()
            optim.step()

            # Track metrics
            train_loss.append(loss.detach().cpu().item())
            train_accuracy.extend((y_pred == y).detach().cpu().tolist())

        # Aggregate training results
        history['train']['loss'].append(torch.mean(torch.tensor(train_loss)).item())
        history['train']['accuracy'].append((torch.sum(torch.tensor(train_accuracy)) / len(train_accuracy)).item())

        ### Validation ###
        model.train(False)
        model.requires_grad_(False)

        # Track across a single epoch
        valid_loss = []
        valid_accuracy = []

        for b, (X, y) in enumerate(valid_dl):
            X, y = X.to(device), y.to(device)

            # Forward pass
            logits = model.forward(X)
            loss: Tensor = loss_fn(logits, y)
            y_pred: Tensor = logits.argmax(dim=1)

            # Track metrics
            valid_loss.append(loss.detach().cpu().item())
            valid_accuracy.extend((y_pred == y).detach().cpu().tolist())

        # Aggregate training results
        history['valid']['loss'].append(torch.mean(torch.tensor(valid_loss)).item())
        history['valid']['accuracy'].append((torch.sum(torch.tensor(valid_accuracy)) / len(valid_accuracy)).item())

        # Inform regarding current metrics
        print('t_loss: %f, t_acc: %f, v_loss: %f, v_acc: %f'
              % (history['train']['loss'][-1], history['train']['accuracy'][-1], history['valid']['loss'][-1], history['valid']['accuracy'][-1]))

    # Output the obtained results so far
    return history

In [8]:
# Q: Train the model for 2 epochs using a cross-entropy loss and an Adam optimizer with a lr of 1e-4
# Prepare training settings
epochs = 2
lr_rate = 1e-4
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=lr_rate)

# Send model to GPU
model = model.to(device)

# Start training
history = train_validate(
    model=model,
    train_dl=train_dl,
    valid_dl=valid_dl,
    epochs=epochs,
    loss_fn=loss_fn,
    optim=optim,
)

Epoch [1/2] - t_loss: 3.627621, t_acc: 0.232642, v_loss: 2.983734, v_acc: 0.342166
Epoch [2/2] - t_loss: 2.666375, t_acc: 0.415298, v_loss: 2.404627, v_acc: 0.468894


## Experiments:

1. Rerun training (restart kernel and run all cells) but this time, when loading the model in the first block of code, specify 'pretrained = True' in order to make use of the weights pretrained on Imagenet.
2. Rerun the code using the pretrained model but this time use a learning rate of 1e-3. What happens?
3. Rerun using the pretrained model and a lr of 1e-4 but this time only change the last layer in the model instead of the entire classifier.
3. Rerun the code using the pretrained model and a lr of 1e-4. This time, freeze the pretrained layers and only update the new layers for the first epochs. Afterwards, proceed to update the entire model. You can freeze parameters by specifying 'requires_grad = False'.
4. Rerun experiment 3 but gradually unfreeze layers instead of unfreezeing the entire model at once.

### Experiment 1

1. Rerun training (restart kernel and run all cells) but this time, when loading the model in the first block of code, specify 'pretrained = True' in order to make use of the weights 

In [56]:
from torchvision.models import AlexNet_Weights
from torchvision.transforms.v2 import Transform


# Use original transformations of AlexNet
weights = AlexNet_Weights.DEFAULT
preprocess: Transform = weights.transforms()

# Preprocess the dataset using those transforms
dataset = torchvision.datasets.Caltech101(
    './dataset',
    transform = Lambda(lambda x: preprocess(x))
)

# Redefine subsets & dataloaders
train_ds, val_ds, test_ds = random_split(dataset, [0.8, 0.1, 0.1], gen)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True, generator=gen)
valid_dl = DataLoader(val_ds, batch_size=batch_size, shuffle=True, generator=gen)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True, generator=gen) ## todo define custom collate function

In [58]:
# Use pretrained model
model = torchvision.models.alexnet(weights=weights)

# Create a new classifier similar to AlexNet
model.classifier = torch.nn.Sequential(
    Dropout(p=0.5, inplace=False),
    Linear(in_features=9216, out_features=4096, bias=True),
    ReLU(inplace=True),
    Dropout(p=0.5, inplace=False),
    Linear(in_features=4096, out_features=4096, bias=True),
    ReLU(inplace=True),
    Linear(in_features=4096, out_features=101, bias=True)
)

# Prepare training settings
epochs = 2
lr_rate = 1e-4
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=lr_rate)

# Send model to GPU
model = model.to(device)

# Start training
history = train_validate(
    model=model,
    train_dl=train_dl,
    valid_dl=valid_dl,
    epochs=epochs,
    loss_fn=loss_fn,
    optim=optim,
)

Epoch [1/2] - 

RuntimeError: output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]

### Experiment 2

In [ ]:
# Use pretrained model
model = torchvision.models.alexnet(weights=weights)

# Create a new classifier similar to AlexNet
model.classifier = torch.nn.Sequential(
    Dropout(p=0.5, inplace=False),
    Linear(in_features=9216, out_features=4096, bias=True),
    ReLU(inplace=True),
    Dropout(p=0.5, inplace=False),
    Linear(in_features=4096, out_features=4096, bias=True),
    ReLU(inplace=True),
    Linear(in_features=4096, out_features=101, bias=True)
)

# Prepare training settings
epochs = 2
lr_rate = 1e-4
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=lr_rate)

# Send model to GPU
model = model.to(device)

# Start training
history = train_validate(
    model=model,
    train_dl=train_dl,
    valid_dl=valid_dl,
    epochs=epochs,
    loss_fn=loss_fn,
    optim=optim,
)